In [49]:
import sys
import time
import datetime as dt
from datetime import datetime
import urllib
# Number manipulation
import numpy as np


# Structure manipulation
import pandas as pd
from pandas import Series, DataFrame
from pandas.tools.plotting import scatter_matrix


# Graphing 
import seaborn as sb

import matplotlib.pyplot as plt
from matplotlib import rcParams


# Stats
import scipy as sp
from scipy import stats
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import chi2_contingency


# Machine Learning & Preprocessing Package
import sklearn
import sklearn.metrics as sm
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LassoLarsCV
from sklearn.linear_model import LassoLarsIC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
from sklearn import decomposition
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cross_validation import train_test_split

import statsmodels.api as sm
import statsmodels.formula.api as smf

from collections import Counter

import networkx as nx



# Parameters for graphs
%matplotlib inline
rcParams['figure.figsize'] = 10,10
sb.set_style('whitegrid')


# Mac
#x = pd.read_csv('/Users/magicsoccer10/Dropbox/twerk werk/Property Value/07-17.csv')

# Arch
#x = pd.read_csv('C:/Users/scott/Dropbox/twerk werk/Data/cre_values.csv')

# Work
x = pd.read_csv('C:/Users/sstandring/Dropbox/twerk werk/Data/cre_values.csv')



epv = pd.read_excel('C:/Users/sstandring/Desktop/Data Entry/Estimating Property Value2.xlsx')

df = x.dropna(axis = 0)
epv.head()

,Property: ID,Prop Sale Cap,Units,Square Footage,Prop Zip Code,Unnamed: 5,Year Built
0,a0F1a000001C7lR,4.5,9,7266,94602,94602,1963
1,a0F1a000001C79B,4.5,3,2130,94702,94702,1954
2,a0F1a000000p4NQ,4.5,44,5000,94609,94609,1951
3,a0F1a000000p2od,4.5,18,37333,94402,94402,1959
4,a0F1a000000p2oe,4.5,8,6374,94080,94080,1977


In [50]:
df['sqft'] = df['Bldg SF']
df['cap'] = df['Cap_Rate'].astype(int)
df['units'] = df['Number Of Units']
df['sale_date'] = pd.to_datetime(df['Sale Date'])
df['sale_quarter'] = df['sale_date'].dt.quarter
df['sale_quarter'] = df['sale_quarter'].astype('str')
df['log_price'] = np.log(df.Sale_Price)
df['log_sqft'] = np.log(df['Bldg SF'])
df['log_far'] = np.log(df['Floor Area Ratio'])
df['log_units'] = np.log(df.units)

num = LabelEncoder()
df['bin_city'] = num.fit_transform(df['Property City'].astype('str'))
df['bin_yrbuilt'] = num.fit_transform(df['Year Built'].astype('str'))

def remover(s):
    return int(s[-4:])

df['year'] = [remover(s) for s in df['Sale Date']]

df['sale_year'] = df['year'].astype('str')
df['sale_y_q'] = df.sale_year + '.' + df.sale_quarter
df['sale_year_qrt'] = df['sale_y_q'].astype('float')
df['sale_year_qrt'] = df['sale_year_qrt'] - 2000

df['yearbuilt'] = df['Year Built'].astype('str').str[:-3]
df['yearbuilt'] = df['yearbuilt'].astype(int)

df['Property Zip Code'] = df['Property Zip Code'].astype('str').str[:-1]
df['bin_zip'] = num.fit_transform(df['Property Zip Code'].astype('str'))
df['zipper'] = df['Property Zip Code'].astype('str')


C:\Users\sstandring\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\sstandring\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\sstandring\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [51]:
epv = epv.dropna(axis = 0)
epv['zipper'] = epv['Unnamed: 5'].astype('str').str[:-1]
epv['yearbuilt'] = epv['Year Built'].astype('str').str[:-1]
epv['yearbuilt'] = epv['yearbuilt'].astype(int)
epv['cap'] = epv['Prop Sale Cap']
epv['units'] = epv['Units']
epv['sqft'] = epv['Square Footage']
epv.head()

,Property: ID,Prop Sale Cap,Units,Square Footage,Prop Zip Code,Unnamed: 5,Year Built,zipper,yearbuilt,cap,units,sqft
0,a0F1a000001C7lR,4.5,9,7266,94602,94602,1963,9460,196,4.5,9,7266
1,a0F1a000001C79B,4.5,3,2130,94702,94702,1954,9470,195,4.5,3,2130
2,a0F1a000000p4NQ,4.5,44,5000,94609,94609,1951,9460,195,4.5,44,5000
3,a0F1a000000p2od,4.5,18,37333,94402,94402,1959,9440,195,4.5,18,37333
4,a0F1a000000p2oe,4.5,8,6374,94080,94080,1977,9408,197,4.5,8,6374


In [52]:
mf = df.loc[df['PropertyType'] == 'Multi-Family']
mf = mf.loc[mf['sale_year'] != '2007']
mf = mf.loc[mf['sale_year'] != '2008']
mf = mf.loc[mf['sale_year'] != '2009']
mf = mf.loc[mf['sale_year'] != '2010']
mf = mf.loc[mf['sale_year'] != '2011']
freqs = Counter(df.zipper)
print(freqs)

grp = mf.groupby(['zipper'])

x=mf.zipper.unique()
cap_mean = []
for i in x:
    zipped = mf.loc[mf['zipper'] == i]
    means = np.mean(zipped.cap)
    cap_mean.append(means)


Counter({'9411': 467, '9460': 430, '9410': 319, '9461': 214, '9511': 213, '9412': 184, '9512': 171, '9470': 163, '9406': 150, '9454': 141, '9408': 110, '9450': 103, '9505': 99, '9480': 92, '9440': 91, '9413': 86, '9404': 84, '9457': 80, '9459': 76, '9452': 73, '9401': 68, '9490': 58, '9402': 45, '9456': 45, '9453': 40, '9455': 39, '9430': 33, '9500': 33, '9400': 32, '9494': 31, '9513': 25, '9451': 24, '9407': 21, '9403': 21, '9496': 19, '9462': 18, '9458': 16, '9471': 13, '9493': 11, '9503': 10, '9492': 9, '9501': 9, '9415': 1})


In [53]:
# formula must have variables in string, no spaces!
est = smf.ols(formula = 'np.log(Sale_Price) ~ (cap + (np.log(units)/np.log(sqft)) + yearbuilt) * C(zipper)', data = mf)
results = est.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     np.log(Sale_Price)   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.904
Method:                 Least Squares   F-statistic:                     114.6
Date:                Thu, 22 Jun 2017   Prob (F-statistic):               0.00
Time:                        14:58:52   Log-Likelihood:                -649.75
No. Observations:                2518   AIC:                             1717.
Df Residuals:                    2309   BIC:                             2936.
Df Model:                         208                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------------------

In [56]:
#'np.log(Sale_Price) ~ cap * (np.log(units)/np.log(sqft)) * sale_year_qrt * yearbuilt * C(zipper)'
a = pd.DataFrame({'cap':[2.88],'units':[6], 'sqft':[9504],'yearbuilt':[195], 'zipper':['9410']})
ans = results.predict(epv)
price = np.exp(ans).astype(int)
ppsf = price/epv.sqft
print('Price: ')
print(price)
print('Price per SQFT: ')
print(ppsf.values)
epv['price'] = price

Price: 
[1350264  911123 2393033 ..., 1047729  934605  902397]
Price per SQFT: 
[ 185.83319571  427.757277    478.6066     ...,  300.03694158  280.83082933
  324.13685345]


In [57]:
epv.head()

,Property: ID,Prop Sale Cap,Units,Square Footage,Prop Zip Code,Unnamed: 5,Year Built,zipper,yearbuilt,cap,units,sqft,price
0,a0F1a000001C7lR,4.5,9,7266,94602,94602,1963,9460,196,4.5,9,7266,1350264
1,a0F1a000001C79B,4.5,3,2130,94702,94702,1954,9470,195,4.5,3,2130,911123
2,a0F1a000000p4NQ,4.5,44,5000,94609,94609,1951,9460,195,4.5,44,5000,2393033
3,a0F1a000000p2od,4.5,18,37333,94402,94402,1959,9440,195,4.5,18,37333,5107607
4,a0F1a000000p2oe,4.5,8,6374,94080,94080,1977,9408,197,4.5,8,6374,2099000


In [60]:
writer = pd.ExcelWriter('C:/Users/sstandring/Desktop/Data Entry/estimated_values.xlsx', engine = 'xlsxwriter')

epv.to_excel(writer, sheet_name='Sheet1')

writer.save()